In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

import requests
import json

import pybaseball
from pybaseball import statcast
pybaseball.cache.enable()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
# 1. get pitch level data from statcast (call this df)

def get_pitch_level_data(season):
    '''
    get the pitch level data from statcast
    *only regular season data*
    '''
    date = {
        2014:("2014-03-22","2014-09-28"),
        2015:("2015-04-05","2015-10-04"),
        2016:("2016-04-03","2016-10-02"),
        2017:("2017-04-02","2017-10-01"),
        2018:("2018-03-29","2018-09-30"),
        2019:("2019-03-20","2019-09-29"),
        2020:("2020-07-23","2022-09-27"),
        2021:("2021-04-01","2021-10-03"),
        2022:("2022-04-07","2022-10-05"),
        2023:("2023-03-30","2023-10-01"),
        2024:("2024-03-20","2024-09-29"),
    }
    df = statcast(start_dt=date[season][0],end_dt=date[season][1]).reset_index().sort_index(ascending=False).reset_index(drop=True)
    return df

# ---------------------------------------------------------------------------------------------------------------------------------

# 2. for each df (of a season), create a dataset that contains basic matchup information of the season (without player stats)

def get_away_batting_order(group):
    away_batters = group.loc[group['inning_topbot'] == 'Top', 'batter'].unique()[:9]
    away_b_dict = {f'away_b{i+1}':away_batters[i] for i in range(9)}
    return pd.Series(away_b_dict)

def get_home_batting_order(group):
    home_batters = group.loc[group['inning_topbot'] == 'Bot', 'batter'].unique()[:9]
    home_b_dict = {f'home_b{i+1}':home_batters[i] for i in range(9)}
    return pd.Series(home_b_dict)

def get_game_info(df):
    '''
    df: a raw statcast dataset
    '''
    # list out what kind of data we need for each game
    cols = ['home_result','game_pk','date','away_team','home_team','post_away_score','post_home_score','away_starting_pitcher','home_starting_pitcher']

    # add batters 1-9 for both away and home
    for i in ['away','home']:
        for j in range(1,10):
            cols.append(f'{i}_b{j}')

    data = df.groupby('game_pk',sort=False).apply(lambda group: pd.Series({
        'home_result': None,
        'date': group['game_date'].iloc[0],  # only one corresponding value
        'away_team': group['away_team'].iloc[0],  # only one corresponding value
        'home_team': group['home_team'].iloc[0],  # only one corresponding value
        'away_final_score': group['post_away_score'].iloc[-1],
        'home_final_score': group['post_home_score'].iloc[-1],
        'away_starting_pitcher': group.loc[(group['inning'] == 1) & (group['inning_topbot'] == 'Bot'), 'pitcher'].iloc[0],
        'home_starting_pitcher': group.loc[(group['inning'] == 1) & (group['inning_topbot'] == 'Top'), 'pitcher'].iloc[0],
    })).sort_values(by='date', ascending=True).reset_index()
    
    data['home_result'] = np.where(data['home_final_score'] > data['away_final_score'], 'W', 'L')

    away_batters = df.groupby('game_pk',sort=False).apply(get_away_batting_order)
    home_batters = df.groupby('game_pk',sort=False).apply(get_home_batting_order)

    data = data.merge(away_batters, on='game_pk').merge(home_batters, on='game_pk')
    
    return data

# ---------------------------------------------------------------------------------------------------------------------------------

# 3. get batting stats

def extract_batting_events(group):
    ab_events = ['single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 
                 'grounded_into_double_play', 'double_play', 'triple_play', 'strikeout_double_play',
                 'fielders_choice', 'fielders_choice_out', 'field_error', 'force_out']
    
    ab_count = group[group['events'].isin(ab_events)].shape[0]
    bb_count = group[group['events'] == 'walk'].shape[0]
    hbp_count = group[group['events'] == 'hit_by_pitch'].shape[0]
    single_count = group[group['events'] == 'single'].shape[0]
    double_count = group[group['events'] == 'double'].shape[0]
    triple_count = group[group['events'] == 'triple'].shape[0]
    home_run_count = group[group['events'] == 'home_run'].shape[0]
    sac_fly_count = group[group['events'].isin(['sac_fly', 'sac_fly_double_play'])].shape[0]

    return pd.Series({
        'ab': ab_count,
        'bb': bb_count,
        'hbp': hbp_count,
        'single': single_count,
        'double': double_count,
        'triple': triple_count,
        'hr': home_run_count,
        'sf': sac_fly_count
    })

def add_batting_order(df):
    df.insert(2,'batting_order',df.index%9+1)
    return df

def get_batting_stats(df):
    '''
    df: a raw statcast dataset
    '''

    # get game info first so its easier to sort batting stats according to game timeline 
    game = get_game_info(df)
    game_away = game[['game_pk','away_team']].rename(columns = {'away_team':'team'})
    game_home = game[['game_pk','home_team']].rename(columns = {'home_team':'team'})
    game_away.insert(0,'game_sequence',game_away.index+1)
    game_home.insert(0,'game_sequence',game_home.index+1)
    game_away['is_home'] = False
    game_home['is_home'] = True

    # batting stats 
    batting_away = (df[df['inning_topbot']=='Top']
                    .groupby(['game_pk','batter'],sort=False)
                    .apply(extract_batting_events))
    batting_home = (df[df['inning_topbot']=='Bot']
                    .groupby(['game_pk','batter'],sort=False)
                    .apply(extract_batting_events))
    
    batting_away_lineup = batting_away.groupby('game_pk').head(9).reset_index()
    batting_home_lineup = batting_home.groupby('game_pk').head(9).reset_index()

    batting_away_lineup_with_order = add_batting_order(batting_away_lineup)
    batting_home_lineup_with_order = add_batting_order(batting_home_lineup)

    data_away = pd.merge(game_away,batting_away_lineup_with_order,on='game_pk')
    data_home = pd.merge(game_home,batting_home_lineup_with_order,on='game_pk')

    data = (pd.concat([data_away,data_home],axis=0)
            .sort_values(by=['game_sequence','is_home'])
            .reset_index(drop=True))
    
    return data

# ---------------------------------------------------------------------------------------------------------------------------------

# 4. get pitching data 
# convert innings pitched (string) into float
def convert_ip_to_float(ip):
    whole, fraction = ip.split('.')
    if fraction == '1':
        return float(whole) + 0.33
    elif fraction == '2':
        return float(whole) + 0.67
    else:
        return float(ip)
    
def detect_missing(data):
    if isinstance(data, str):
        if "-" in data:
            return True
    return False

# get data via the MLB Stats API
def extract_p_stats_from_api(game_pk, pitcher_id, is_home = True):
    # URL for the MLB Stats API to get the boxscore of the game
    url = f'https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore'

    # Make a request to the API
    response = requests.get(url)

    if is_home:
        home_away = "home"
    else: 
        home_away = "away"

    if response.status_code == 200:
        # Load the JSON data
        boxscore_data = response.json()
        pitchers = boxscore_data['teams'][home_away]['players']
        for player_id, player_data in pitchers.items():
            if player_id == f'ID{pitcher_id}' and (
                player_data['position']['code'] == '1'  # Code '1' corresponds to pitchers
                or 
                player_data['position']['code'] == '10'  # for Shohei Ohtani :) also his id is 660271
            ):
                name = player_data['person']['fullName']
                stats = player_data['stats']['pitching']
                if len(stats)!=0:
                    n_pitches = stats["pitchesThrown"]
                    ip = convert_ip_to_float(stats["inningsPitched"])
                    er = float(stats["earnedRuns"])
                    k = stats["strikeOuts"]
                    bb = stats["baseOnBalls"]
                    h = stats["hits"]
                    return np, ip, er, k, bb, h
                else:
                    return 0, 0, 0, 0, 0, 0
    else:
        return 0, 0, 0, 0, 0, 0
    

def get_pitching_stats(df): 
    '''
    only get the stats from starting pitchers 
    not considering bullpen
    '''

    # get game info first so its easier to sort batting stats according to game timeline 
    game = get_game_info(df)
    game_away = game[['game_pk','away_team','away_starting_pitcher']].rename(columns = {'away_team':'team','away_starting_pitcher':'starting_pitcher'})
    game_home = game[['game_pk','home_team','home_starting_pitcher']].rename(columns = {'home_team':'team','home_starting_pitcher':'starting_pitcher'})
    game_away.insert(0,'game_sequence',game_away.index+1)
    game_home.insert(0,'game_sequence',game_home.index+1)
    game_away['is_home'] = False
    game_home['is_home'] = True

    # get away starting pitcher data
    pitching_away_stats = pd.DataFrame(columns=['game_pk', 'starting_pitcher', 'np', 'ip', 'er', 'k', 'bb', 'h'])
    for idx, row in game.iterrows():
        game_pk = row['game_pk']
        starting_pitcher = row['starting_pitcher']
        np, ip, er, k, bb, h = extract_p_stats_from_api(game_pk,starting_pitcher, is_home=False)
        new_row = pd.DataFrame([[game_pk,starting_pitcher,np,ip,er,k,bb,h]], columns=pitching_away_stats.columns)
        pitching_away_stats = pd.concat([pitching_away_stats, new_row], ignore_index=True)

    # get home starting pitcher data
    pitching_home_stats = pd.DataFrame(columns=['game_pk', 'starting_pitcher', 'np', 'ip', 'er', 'k', 'bb', 'h'])
    for idx, row in game.iterrows():
        game_pk = row['game_pk']
        starting_pitcher = row['starting_pitcher']
        np, ip, er, k, bb, h = extract_p_stats_from_api(game_pk,starting_pitcher, is_home=True)
        new_row = pd.DataFrame([[game_pk,starting_pitcher,np,ip,er,k,bb,h]], columns=pitching_home_stats.columns)
        pitching_home_stats = pd.concat([pitching_home_stats, new_row], ignore_index=True)

    data_away = pd.merge(game_away,pitching_away_stats,on='game_pk')
    data_home = pd.merge(game_home,pitching_home_stats,on='game_pk')

    data = (pd.concat([data_away,data_home],axis=0)
            .sort_values(by=['game_sequence','is_home'])
            .reset_index(drop=True))
    
    return data
    
# ---------------------------------------------------------------------------------------------------------------------------------

# 5. generate csv files 
def game_info_to_csv(dfs):
    '''
    dfs: a dictionary with key = df_{season} and value = statcast raw data
    get the basic game information with get_game_info
    and generate a csv file for each dataframe
    '''
    for key, value in dfs.items():
        data = get_game_info(value)
        data.to_csv(f'data_2014_to_2024/{key}_game_info.csv')

def batting_stats_to_csv(dfs):
    for key, value in dfs.items():
        data = get_batting_stats(value)
        data.to_csv(f'data_2014_to_2024/{key}_batting_stats.csv')

def pitching_stats_to_csv(dfs):
    for key, value in dfs.items():
        data = get_pitching_stats(value)
        data.to_csv(f'data_2014_to_2024/{key}_pitching_stats.csv')


In [3]:
%%time
# get pitch level data from season 2014 to 2024
dfs = {}
seasons = np.arange(2014,2025)
for season in seasons:
    if season == 2020: 
        continue
    else:
        this_df = get_pitch_level_data(season)
        dfs[f'df_{season}'] = this_df
        game_count = this_df['game_pk'].nunique()
        print(f'game count of season {season}: {game_count}')

This is a large query, it may take a moment to complete


100%|██████████| 191/191 [00:37<00:00,  5.15it/s]


game count of season 2014: 2533
This is a large query, it may take a moment to complete


100%|██████████| 183/183 [00:35<00:00,  5.13it/s]


game count of season 2015: 2429
This is a large query, it may take a moment to complete


...

game count of season 2022: 2430
This is a large query, it may take a moment to complete


100%|██████████| 186/186 [00:46<00:00,  3.99it/s]


game count of season 2023: 2430
This is a large query, it may take a moment to complete


100%|██████████| 194/194 [00:48<00:00,  3.98it/s]


game count of season 2024: 2515
CPU times: user 6min 48s, sys: 10min 5s, total: 16min 54s
Wall time: 9min 53s


In [4]:
%%time
# generate csv files of game info
game_info_to_csv(dfs)

/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = df.groupby('game_pk',sort=False).apply(lambda group: pd.Series({
/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  away_batters = df.groupby('game_pk',sort=False).apply(get_away_batting_order)
/v

CPU times: user 1min 7s, sys: 15.1 s, total: 1min 22s
Wall time: 1min 39s


/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:64: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  home_batters = df.groupby('game_pk',sort=False).apply(get_home_batting_order)


In [5]:
%%time
# generate csv files of batting stats
batting_stats_to_csv(dfs)

/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = df.groupby('game_pk',sort=False).apply(lambda group: pd.Series({
/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  away_batters = df.groupby('game_pk',sort=False).apply(get_away_batting_order)
/v

CPU times: user 1h 46min 19s, sys: 1min 17s, total: 1h 47min 37s
Wall time: 1h 49min 4s


/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:123: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(extract_batting_events))


In [6]:
%%time
# generate csv files of pitching stats
pitching_stats_to_csv(dfs)

/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = df.groupby('game_pk',sort=False).apply(lambda group: pd.Series({
/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_92129/1761772045.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  away_batters = df.groupby('game_pk',sort=False).apply(get_away_batting_order)
/v

KeyError: 'starting_pitcher'